In [ ]:
import torch
import torch.nn as nn
import numpy as np
from skimage import io
import os
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import transforms
import torch.optim as optim
from PIL import Image
import torchvision.datasets as dset
import torchvision
from torchvision import utils, models
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import tensorflow as tf

#hyperparameters
in_channel=3
num_classes=200
learning_rate=1e-3
batch_size=1
num_epochs=1

train_CNN = False

#train_dataset_path=r'C:/Users/Jim/Desktop/HW1/train'
train_dataset_path=r'/content/drive/MyDrive/Hw1/train/'
test_dataset_path=r'/content/drive/MyDrive/Hw1/test/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train_transform = transforms.Compose(
        [
        transforms.Resize(224),  
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )
test_transform = transforms.Compose(
        [
        transforms.Resize(224),  
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )
class MyDataset(Dataset):
    def __init__(self, txt_path, root , transform=train_transform):
        self.annotations = pd.read_csv(txt_path, sep=" ", header=None)
        self.root_dir = root
        self.transform = transform
    def __getitem__(self, index):
        image = self.annotations.iloc[index, 0]

        image_PIL = Image.open(os.path.join(self.root_dir, image)).convert("RGB")

        #y_label=torch.LongTensor(int(self.annotations.iloc[index, 1]))
        
        y_label=torch.LongTensor(int(self.annotations.iloc[index, 1]))


        y_onehot = torch.zeros(num_classes).scatter_(1, y_label ,1)
        
        print(y_onehot)
        if self.transform:
                img = self.transform(image_PIL)     
        return (img , y_label)
    def __len__(self):
        
        return len(self.annotations)

In [ ]:
train_dataset=MyDataset(txt_path=r'/content/drive/MyDrive/Hw1/training_labels.txt',root=train_dataset_path, transform=train_transform)
test_dataset=MyDataset(txt_path=r'/content/drive/MyDrive/Hw1/testing_img_order.txt',root=test_dataset_path, transform=test_transform)

train_loader= torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size , shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size , shuffle=True)

print(train_dataset[0][1].shape)

FileNotFoundError: ignored

In [ ]:
class CNN(nn.Module):
    def __init__(self, train_CNN=False, num_classes=200):
        super(CNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True, aux_logits=False)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, images):
        features = self.inception(images)

        data=self.sigmoid(self.dropout(self.relu(features))).squeeze()
        return data
        #return self.sigmoid(self.dropout(self.relu(features)))

In [ ]:
model = CNN().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


for name, param in model.inception.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN


for epoch in range(num_epochs):
    losses=[]    
    for batch_idx, (data, target) in enumerate(train_loader):
       

        data=data.to(device=device)
        
        target=target.to(device=device)
        
        inputs = model(data)

        loss=criterion(inputs, target)
        
        losses.append(loss.item())
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
       # print(f'cost at epoch {epoch} is {sum(losses)/len(losses)} ')
        
  
def check_accuracy(loader, model):
    if loader == train_loader:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on validation data")
        
        num_correct =0
        num_samples =0
        model.eval()
        
        with torch.no_grad():
            for x, y in loader:
                x=x.to(device=device)
                y=y.to(device=device)
                
                
            scores = model(x)
            predictions = torch.tensor([1.0 if i >= 0.5 else 0.0 for i in scores]).to(device)
             
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
       
                
        return f"{float(num_correct)/float(num_samples)*100:.2f}"
        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )   
           
model.train()
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

194


IndexError: ignored